In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pathlib import Path
import os

# from https://github.com/hauser-group/odft_tools
from odft_tools.layers import (
    IntegrateLayer,
    Continuous1DConvV1,
    Continuous1DConvV2
)

from odft_tools.models import (
    ResNetContConv1DModel,
    ResNetConv1DModel
)

data_path = '../datasets/orbital_free_DFT/'

# Load dataset
Both data .hdf5-Files can be downloaded from https://github.com/hauser-group/datasets/tree/master/orbital_free_DFT

### Training

In [ ]:
with h5py.File(data_path + 'M=100_training_data.hdf5', 'r') as f:
    keys = f.keys()
    print(keys)
    # build a dict (dataset.value has been deprecated. Use dataset[()] instead.)
    data = {key:f[key][()] for key in keys}

In [ ]:
x = np.linspace(0, 1, 500)
dx = x[1] - x[0]
N = 1
# density is wavefunction squared
n = np.sum(data['wavefunctions'][:, :, :N]**2, axis=-1)
# integrate using trapezoidal rule:
V = np.sum(0.5*(data['potential'][:, :-1]*n[:, :-1] 
                + data['potential'][:, 1:]*n[:, 1:])           
           * dx, axis=-1)
# kinetic energy is total energy minus potential energy
T = np.sum(data['energies'][:, :N], axis=-1) - V
# kinetic energy derivative
dT_dn = np.expand_dims(np.sum(data['energies'][:, :N], axis=-1)/N, axis=-1) - data['potential']
n = n.reshape((-1, 500, 1))

### Test Set

In [ ]:
with h5py.File(data_path + 'test_data.hdf5', 'r') as f:
    keys = f.keys()
    print(keys)
    # build a dict (dataset.value has been deprecated. Use dataset[()] instead.)
    data_test = {key:f[key][()] for key in keys}

In [ ]:
# density is wavefunction squared
n_test = np.sum(data_test['wavefunctions'][:, :, :N]**2, axis=-1)
# integrate using trapezoidal rule:
V_test = np.sum(0.5*(data_test['potential'][:, :-1]*n_test[:, :-1] 
                + data_test['potential'][:, 1:]*n_test[:, 1:])           
                * dx, axis=-1)
# kinetic energy is total energy minus potential energy
T_test = np.sum(data_test['energies'][:, :N], axis=-1) - V_test
# kinetic energy derivative
dT_dn_test = - data_test['potential'] + np.expand_dims(np.sum(data_test['energies'][:, :N], axis=-1)/N, axis=-1) 
n_test = n_test.reshape((-1, 500, 1))

In [ ]:
def plot_weigts(model, result_type, before_after):
    if not os.path.exists('results' + result_type):
        os.makedirs('results' + result_type)

    weigths = model.layers[0].get_weights()[0]
    plt.ylabel('density')
    plt.xlabel('kernel size')
    plt.title('Gaussian Kernel of ContConv1V1 with Layer softplus act. fun' + before_after)
    plt.plot(weigths[:, 0, :])
    plt.savefig('results' + result_type + 'gauss_kernel_softplus_act_V1_first_' + before_after + '.png')
    plt.show()

    weigths = model.layers[1].get_weights()[0]
    plt.ylabel('density')
    plt.xlabel('kernel size')
    plt.title('Gaussian Kernel of ContConv1V1 with Layer linear act. fun_' + before_after)
    plt.plot(weigths[:, 0, :])
    plt.savefig('results' + result_type + 'gauss_kernel_linear_act_V1_sec_' + before_after + '.png')
    plt.show()
    
def plot_der_energy(x, dT_dn, model, n, result_type):
    if not os.path.exists('results' + result_type):
        os.makedirs('results' + result_type)

    plt.plot(x, dT_dn[0])
    plt.plot(x, tf.squeeze(model(n[0].reshape((1, 500, 1)).astype(np.float32))['dT_dn']))
    plt.ylabel('dT_dn')
    plt.title('Comparison reference with trained')
    plt.savefig('results' + result_type + 'dT_dn_V1_' + before_after + '.png')
    plt.show()

import os
if not os.path.exists(directory):
    os.makedirs(directory)# Define model

In [ ]:
kernel_size = 100
mean = 5
stddev = 5

res_net_blocks_couts = [1, 2, 5]
epochs = [1, 50, 1000]
training_dataset = tf.data.Dataset.from_tensor_slices((n.astype(np.float32), {'T': T.astype(np.float32), 'dT_dn': dT_dn.astype(np.float32)})).batch(100).repeat(10)

for num_res_nat_blocks in res_net_blocks_couts:
    for epoch in epochs:
        model = ResNetConv1DModel(filter_size=32, kernel_size=100, layer_size=None, num_res_nat_blocks=4, weights_gaus=[5, 5], n_outputs=None, random_init=True, add_cont_layer=True, dx=1)

        model.build(input_shape=(None, 500, 1))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                      loss={'T': 'mse', 'dT_dn': 'mse'}, 
                      loss_weights={'T': 0.2, 'dT_dn': 1.0}, # As recommended by Manuel: scale the loss in T by 0.2
                      metrics={'T': ['mae'], 'dT_dn': ['mae']})
        print('--------------------------------->Start<---------------------------------')
        print(f'No Cont Layer. res_net {num_res_nat_blocks} with {epoch}')
        result_type = '/ResNetConv1D/Conv/res_net_' + str(num_res_nat_blocks) + '_epoch_' + str(epoch) + '/'
        
        plot_weigts(model, result_type, 'before')
        model.summary()
        model.fit(training_dataset, epochs=epoch, verbose=2, validation_data=(n_test, {'T': T_test, 'dT_dn': dT_dn_test}), validation_freq=10)
        plot_weigts(model, result_type, 'after')
        print('--------------------------------->END<---------------------------------')
